In [1]:
#**********************************************************************************
#Fuction to create cross-sections using ICON native grid
#
#Julian Quimbayo Duarte
#Goethe Uni, 2022
#quimbayo-duarte@iau.uni-frankfurt.de
#**********************************************************************************
#--modules
import time
import xarray as xr

import numpy as np
import numpy.ma as ma

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from   matplotlib.collections import PolyCollection

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from   cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader

import math
import csv  

from scipy import interpolate
from scipy.interpolate import griddata
import sys
import datetime as dt

# First Party
sys.path.append('../utilities_tlezuo/')
# from timefunctions import *
import varfunctions as vf
import locfunctions as lf
import timefunctions as tf
import get_stations_timeseries as gs

In [2]:

#*************************************************************************************************
#-- function to get the closer gp to coordinate
def get_indices(lats,lons,radius,LATX , LONX ):
    """ get_indices takes latitude latx and longitude lonx and
    returns the indices indexi and indexj of the nearest grid cell in a 2D array"""
    DISTANCE_TO_X = np.sqrt((lats - LATX)**2 + (lons - LONX)**2)
    indices = np.where(DISTANCE_TO_X < radius )
    return indices[0]
    pass
#*************************************************************************************************
#-- GEt the distance between two points
def haversine(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia
#*************************************************************************************************
#-- Getting the cross-section from the native ICON grid (only one interpolation)
#-- initial and final point for the cross-section

def cross_section(A, B, Step, with_file, icon_gp_file,clon,clat,dx,topo,var,z,interp_meth):
    #-- def some arrays
    X = np.empty(10000)
    Y = np.empty(10000)

    #-- Def the line and points between coordinates (A and B points)
    #-- geo points start and end
    x1, y1 = A
    x2, y2 = B

    #-- the increment step (higher = faster)
    STEP = Step #-- Step between points
    
    # print(str(x1),str(x2))
    if x1 > x2:           # x2 must be the bigger one here
        x1, x2 = x2, x1
        y1, y2 = y2, y1
    #-- getting the diff coordinates
        for i in range(int((x2-x1)/STEP) + 1):
            #print(i)
            x = x1 + i*STEP
            X[i] = x
            y = (y1-y2)/(x1-x2) * (x - x1) + y1
            Y[i] = y
    elif x1 == x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        for i in range(int((y2-y1)/STEP) + 1):
            y = y1 + i*STEP
            Y[i] = y
            x = x1
            X[i] = x
        
    #-- storing the coordinates in an array
    coords_in_line = np.empty([i+1,2])
    coords_in_line[:-1,1] = X[:i]
    coords_in_line[:-1,0] = Y[:i]
#
    #-- cross section lat lon points
    coords_in_line[-1,1] = B[0]
    coords_in_line[-1,0] = B[1]
    print('length of points array is '+str(len(coords_in_line)))

    #-- saving the points to check them in G earth
    np.savetxt('line_coord.csv', coords_in_line, delimiter=",")
    
    if with_file == 'TRUE':
    #-- Reading the files if it is provided
        with open(icon_gp_file) as file_name:
            points_icon = np.loadtxt(file_name, delimiter=",")
    
        ccoords = np.column_stack((clon,clat))
        points_icon_coords = np.empty_like(coords_in_line)
    
    else:
    #-- Getting the closer points in ICON grid
    #-- def some arrays
        ccoords = np.column_stack((clon,clat))
        dist = np.empty_like(clon)
        points_icon = np.empty_like(coords_in_line)
        points_icon_coords = np.empty_like(coords_in_line)
        topo_icon = np.empty_like(coords_in_line)
#
        jj = 0
        for j in coords_in_line:
            ii=0
            for i in ccoords:
                dist[ii] = haversine(i[1],i[0],j[1],j[0]) #km
                ii=ii+1
            indices = np.where(dist < dx )  #.26 grid resolution
            bb = np.argmin(dist[indices])
            indices = np.squeeze(np.asanyarray(indices))
            points_icon[jj] = indices[bb] #Closer points in Icon
            jj=jj+1
            # print(jj)
        np.savetxt(icon_gp_file, points_icon, delimiter=",")   

        #-- Obtain the coordinates in ICON + the values for the diff sfc quantities

    topo_icon = np.empty_like(coords_in_line)
    ii = 0
    for i in (points_icon[:,0]):
        points_icon_coords[ii,:] = (ccoords[int(i)]) #Coordinates in ICON
        topo_icon[ii,:] = topo[int(i)] #Topo values for that coordinates
        ii = ii+1

    np.savetxt('grid_coord.csv', points_icon_coords, delimiter=",")
    
    #-- Def new x-array to plot the data

    cross_x = np.empty_like(coords_in_line)
    dim_coords_in_line = coords_in_line.shape 

    for i in range(0,dim_coords_in_line[0]-1):
        cross_x[i,0] = haversine(coords_in_line[i,1], coords_in_line[i,0],
                               coords_in_line[i+1,1], coords_in_line[i+1,0])
        cross_x[i,1] = sum(cross_x[0:i,0])
    #-- last item
    cross_x[-1,1] = sum(cross_x[0:i+1,0])
    #

    #-- Interpolation (regridding) for sfc data
    zz = topo_icon[:,0]
    ter = griddata(points_icon_coords, zz, coords_in_line, method=interp_meth)    
    
    #-- Interpolation 3D variables

    #-- Def some arrays
    var_icon = np.empty_like(coords_in_line)
    dim_var_icon = var_icon.shape
    dim_var = var.shape
    regrid_var = np.empty((dim_var[0],dim_var_icon[0]))
    
    z_icon = np.empty_like(coords_in_line)
    regrid_z = np.empty((dim_var[0],dim_var_icon[0]))

    for k in range(0,dim_var[0]-1):
        ii = 0
        for i in (points_icon[:,0]):
            points_icon_coords[ii,:] = (ccoords[int(i)])
            var_icon[ii,:] = var[dim_var[0]-1-k,int(i)]
            z_icon[ii,:] = z[dim_var[0]-1-k,int(i)]
            ii = ii+1
    
        var_new = var_icon[:,0]
        zz = z_icon[:,0]
    
        regrid_var[k,:] = griddata(points_icon_coords, var_new, coords_in_line, method=interp_meth)
        regrid_z[k,:] = griddata(points_icon_coords, zz, coords_in_line, method=interp_meth)/1000 # in km
    return regrid_var,regrid_z,ter,cross_x
# 3d var, vertical grid, 2d var, horizontal grid
#*************************************************************************************************

In [3]:
## GET UNIVERSAL DATA ##
# extfile: get topography and latlon grid
extfile = xr.open_dataset('/store/s83/tlezuo/external_parameter_mch_ICON_1E_R19B08_DOM1.nc')
topo_CTRL = extfile['topography_c'].values
clon_CTRL = np.rad2deg(extfile.clon.values)
clat_CTRL = np.rad2deg(extfile.clat.values)
# constfile: get z levels
constfile = xr.open_dataset('/store/s83/tlezuo/RUN2_reference/lfff00000000c.nc')
z_CTRL = np.squeeze(constfile['HHL'].values)

In [5]:
## DECIDE ##
# cross section
line=lf.VCS_lidars1
# variable
pvar = vf.W
# time
pdate = dt.datetime(2019,9,13,15,00)


length of points array is 13
length of points array is 13


In [8]:
################################# DEFINING CS #######################################
# cross section metadata
A = [line.latmin, line.lonmin]
B = [line.latmax, line.lonmax]
icon_gp_file = 'icon_ptos_1k_'+line.short+'.csv' # file name from which to load grid
Step = 0.0025 #about 250 m
interp_meth = 'cubic'
dx_CTRL = 1

# standard nc file
lt = tf.get_lt(pdate, dt.datetime(2019,9,12,12,00))
filename = tf.lfff_name(lt)
ncfile = xr.open_dataset('/store/s83/tlezuo/RUN2_reference/out_std/'+filename)
pvar_array = np.squeeze(ncfile[pvar.name].values)
th_array = np.squeeze(ncfile['T'].values)*(100000/(np.squeeze(ncfile['P'].values)))**(2/7)

################################# GET CS #######################################
# get variable 
regrid_CTRL = cross_section(A, B, Step, 'TRUE', icon_gp_file,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,pvar_array,z_CTRL,interp_meth)
regrid_pvar = regrid_CTRL[0]

# get TH for contour
regrid_CTRL = cross_section(A, B, Step, 'TRUE', icon_gp_file,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,th_array,z_CTRL,interp_meth)
regrid_temp_ctrl = regrid_CTRL[0]
regrid_z_ctrl = regrid_CTRL[1]
ter_ctrl = regrid_CTRL[2]
cross_x_ctrl = regrid_CTRL[3]
dim_theta = th_array.shape

################################# PLOTTING #######################################
fig,ax=plt.subplots(figsize=(9, 4))

#-- def x-2d array #As all are regridded in the same grid, we only define this once for all plots
x_vert = np.linspace(0, dim_theta[0]-1, dim_theta[0])
xx = np.meshgrid(cross_x_ctrl[:,1], x_vert)
aa =np.asarray(xx[0])

levels_th = range(280, 320, 1)
levels_pvar = np.arange(pvar.min,pvar.max,pvar.ticks)
# levels_pvar = np.arange(-10,11,2)
fontsize = 12
#-- plotting 
if pvar.name == 'W':
    pvar_plot = ax.contourf(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_pvar[:-2,1:]
                        ,extend='both',levels=levels_pvar,cmap=pvar.cmap)
else:
    pvar_plot = ax.contourf(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_pvar[:-1,1:]
                        ,extend='both',levels=levels_pvar,cmap=pvar.cmap)
th_plot = ax.contour(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_temp_ctrl[:-1,1:]
                     ,levels=levels_th,colors='k',linewidths=0.30)
ax.clabel(th_plot, levels_th[::2], inline=1, fontsize=10)
#--plot terrain
ter_line = ax.plot(aa[0,:-1],(ter_ctrl[1:]+10)/1000,color='k',linewidth=2) #the +10 is because of the vertical grid resolution

#-- Colorbar
cbar = fig.colorbar(pvar_plot)
cbar.ax.tick_params(labelsize='large')
cbar.set_label(pvar.name+pvar.units, rotation=90,fontsize=fontsize)

#-- axis settings
# ax.set_xlim(0,line.dist)
ax.set_ylim(0,3)
plt.setp(ax.get_xticklabels(),fontsize=fontsize)
plt.setp(ax.get_yticklabels(),fontsize=fontsize)

#-- titles
ax.set_ylabel('Height [km]',fontsize=fontsize)
ax.set_xlabel('Distance [km]',fontsize=fontsize)
ax.set_title(pvar.title + line.name,fontsize=fontsize)

#-- Pic frame
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
    
#-- maximize and save the pdf file
# fig_name = pvar.name + line.short + pdate.strftime('%d%m%y_%H%M')
# plt.savefig(fig_name, bbox_inches='tight',dpi=300)

In [198]:
## CODE TO COMBINE MULTIPLE CS IN ONE PLOT ## LIDAR CS
## DECIDE ##
# variable
pvar = vf.W
# time
pdate = dt.datetime(2019,9,13,15,00)
# cross section
line1=lf.VCS_lidars2
line2=lf.VCS_lidars1

################################# DEFINING CS #######################################
# cross section metadata
A = [line1.latmin, line1.lonmin]
B = [line1.latmax, line1.lonmax]
C = [line2.latmin, line2.lonmin]
D = [line2.latmax, line2.lonmax]
icon_gp_file1 = 'icon_ptos_1k_'+line1.short+'.csv' # file name from which to load grid
icon_gp_file2 = 'icon_ptos_1k_'+line2.short+'.csv' # file name from which to load grid
Step = 0.0025 #about 250 m
interp_meth = 'cubic'
dx_CTRL = 1

# standard nc file
lt = tf.get_lt(pdate, dt.datetime(2019,9,12,12,00))
filename = tf.lfff_name(lt)
ncfile = xr.open_dataset('/store/s83/tlezuo/RUN2_reference/out_std/'+filename)
pvar_array = np.squeeze(ncfile[pvar.name].values)
th_array = np.squeeze(ncfile['T'].values)*(100000/(np.squeeze(ncfile['P'].values)))**(2/7)

################################# GET CS #######################################
# get variable 
regrid_CTRL1 = cross_section(A, B, Step, 'TRUE', icon_gp_file1,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,pvar_array,z_CTRL,interp_meth)
regrid_pvar1 = regrid_CTRL1[0]
regrid_CTRL2 = cross_section(C, D, Step, 'TRUE', icon_gp_file2,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,pvar_array,z_CTRL,interp_meth)
regrid_pvar2 = regrid_CTRL2[0]

# get TH for contour
regrid_CTRL1 = cross_section(A, B, Step, 'TRUE', icon_gp_file1,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,th_array,z_CTRL,interp_meth)
regrid_temp_ctrl1 = regrid_CTRL1[0]
regrid_z_ctrl1 = regrid_CTRL1[1]
ter_ctrl1 = regrid_CTRL1[2]
cross_x_ctrl1 = regrid_CTRL1[3]
regrid_CTRL2 = cross_section(C, D, Step, 'TRUE', icon_gp_file2,clon_CTRL,clat_CTRL,
                            dx_CTRL,topo_CTRL,th_array,z_CTRL,interp_meth)
regrid_temp_ctrl2 = regrid_CTRL2[0]
regrid_z_ctrl2 = regrid_CTRL2[1]
ter_ctrl2 = regrid_CTRL2[2]
cross_x_ctrl2 = regrid_CTRL2[3]
dim_theta = th_array.shape


length of points array is 14
length of points array is 13
length of points array is 14
length of points array is 13


In [258]:
np.isnan(regrid_z_ctrl2).any(axis=0).sum()

5

In [278]:
################################# APPENDING #######################################
# pvar 
regrid_pvar = np.concatenate((regrid_pvar1,regrid_pvar2),axis=1) # concatenating
regrid_pvar = regrid_pvar[:,~np.isnan(regrid_pvar).any(axis=0)] # removing nan
regrid_pvar=np.delete(regrid_pvar,12,axis=1) # removing wird number
print(regrid_pvar.shape)
      
# th 
regrid_temp_ctrl = np.concatenate((regrid_temp_ctrl1,regrid_temp_ctrl2),axis=1) # concatenating
regrid_temp_ctrl = regrid_temp_ctrl[:,~np.isnan(regrid_temp_ctrl).any(axis=0)] # removing nan
# regrid_temp_ctrl=np.delete(regrid_temp_ctrl,12,axis=1) # removing wird number
print(regrid_temp_ctrl.shape)
# z
regrid_z_ctrl = np.concatenate((regrid_z_ctrl1,regrid_z_ctrl2),axis=1) # concatenating
regrid_z_ctrl = regrid_z_ctrl[:,~np.isnan(regrid_z_ctrl).any(axis=0)] # removing nan
regrid_z_ctrl=np.delete(regrid_z_ctrl,12,axis=1) # removing wird number
print(regrid_z_ctrl.shape)

# terrain and x
ter_ctrl = np.concatenate((ter_ctrl1,ter_ctrl2))
cross_x_ctrl2 = cross_x_ctrl1[-1]+cross_x_ctrl2
cross_x_ctrl = np.concatenate((cross_x_ctrl1,cross_x_ctrl2))

# delete nans

nan_index = np.argwhere(~np.isnan(ter_ctrl))
cross_x_ctrl = cross_x_ctrl[nan_index,:][:,0,:]
ter_ctrl=ter_ctrl[~np.isnan(ter_ctrl)]
ter_ctrl=np.delete(ter_ctrl,12)
cross_x_ctrl=np.delete(cross_x_ctrl,12,axis=0)
print(ter_ctrl.shape)
print(cross_x_ctrl.shape)


(81, 20)
(80, 20)
(80, 20)
(20,)
(20, 2)


In [250]:

print(regrid_pvar.shape)
print(regrid_z_ctrl.shape)
print(regrid_z_ctrl.shape)

(80, 20)


In [283]:
################################# PLOTTING #######################################
fig,ax=plt.subplots(figsize=(9, 4))

#-- def x-2d array #As all are regridded in the same grid, we only define this once for all plots
x_vert = np.linspace(0, dim_theta[0]-1, dim_theta[0])
xx = np.meshgrid(cross_x_ctrl[:,1], x_vert)
aa =np.asarray(xx[0])

levels_th = range(280, 320, 1)
levels_pvar = np.arange(pvar.min,pvar.max,pvar.ticks)
# levels_pvar = np.arange(-10,11,2)
fontsize = 12
#-- plotting 
if pvar.name == 'W':
    pvar_plot = ax.contourf(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_pvar[:-2,1:]
                        ,extend='both',levels=levels_pvar,cmap=pvar.cmap)
else:
    pvar_plot = ax.contourf(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_pvar[:-1,1:]
                        ,extend='both',levels=levels_pvar,cmap=pvar.cmap)
th_plot = ax.contour(aa[:-1,:-1],regrid_z_ctrl[:-1,1:],regrid_temp_ctrl[:-1,1:]
                     ,levels=levels_th,colors='k',linewidths=0.30)
ax.clabel(th_plot, levels_th[::2], inline=1, fontsize=10)
#--plot terrain
ter_line = ax.plot(aa[0,:-1],(ter_ctrl[1:]+10)/1000,color='k',linewidth=2) #the +10 is because of the vertical grid resolution

#-- Colorbar
cbar = fig.colorbar(pvar_plot)
cbar.ax.tick_params(labelsize='large')
cbar.set_label(pvar.name+pvar.units, rotation=90,fontsize=fontsize)

#-- axis settings
# ax.set_xlim(0,line.dist)
ax.set_ylim(0,3)
plt.setp(ax.get_xticklabels(),fontsize=fontsize)
plt.setp(ax.get_yticklabels(),fontsize=fontsize)

#-- titles
ax.set_ylabel('Height [km]',fontsize=fontsize)
ax.set_xlabel('Distance [km]',fontsize=fontsize)
ax.set_title(pvar.title + line.name,fontsize=fontsize)

#-- Pic frame
for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(1.5)
    
#-- maximize and save the pdf file
# fig_name = pvar.name + line.short + pdate.strftime('%d%m%y_%H%M')
# plt.savefig(fig_name, bbox_inches='tight',dpi=300)

In [285]:
xx

[array([[  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597],
        [  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597],
        [  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597],
        ...,
        [  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597],
        [  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597],
        [  0.        ,   0.29203132,   0.58406135, ..., 107.24621782,
         107.54870801, 107.85119597]]),
 array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  1.,  1.,  1.],
        [ 2.,  2.,  2., ...,  2.,  2.,  2.],
        ...,
        [77., 77., 77., ..., 77., 77., 77.],
        [78., 78., 78., ..., 78., 78., 78.],
        [79., 79., 79., ..., 79., 79., 79.]])]